In [1]:
import os
import sys

In [2]:
sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from common import polynom_factory

from scipy import linalg

%matplotlib inline
np.set_printoptions(precision=3, linewidth=200)

In [4]:
order = 2
interpolant = np.eye(order+1)

In [5]:
lgg = polynom_factory.local_gradgrad_matrix(dim=2, distribution='globatto', order=order)
lgf = polynom_factory.local_gradfunc_matrix(dim=2, distribution='globatto', order=order)

In [6]:
dof_l_f = list(range((order+1)**2 - (order+1),(order+1)**2))
F_lr = np.dot(interpolant, lgf[1][2][((order+1)**2 - (order+1)):])


dof_r_f = list(range(order+1))
F_rl = np.dot(interpolant, lgf[1][0][:(order+1)])

In [7]:
dist_matrix_r = np.zeros((order+1, (order+1)**2))
for num, dof in enumerate(dof_r_f):
    dist_matrix_r[num, dof] = 1. 

In [8]:
dist_matrix_l = np.zeros((order+1, (order+1)**2))
for num, dof in enumerate(dof_l_f):
    dist_matrix_l[num, dof] = 1. 

In [26]:
upper = np.hstack((lgg[0], np.dot(dist_matrix_l.T, F_rl)))
bottom = np.hstack((np.dot(dist_matrix_r.T, F_lr), lgg[0]))

In [27]:
whole = np.vstack((upper, bottom))

In [28]:
whole_dist = np.zeros((2*(order+1)**2,2*(order+1)**2 - (order+1)))
whole_dist_avg = np.zeros((2*(order+1)**2,2*(order+1)**2 - (order+1)))
whole_dist_map = np.zeros((2*(order+1)**2,2*(order+1)**2))
sum = 0
for i in range((order+1)**2):
    whole_dist[i,i] = 1
    if(i>=((order+1)**2 - (order+1))):
        whole_dist_avg[i,i] = .5
        whole_dist_map[i,i + 1 + order] = .5
        whole_dist_map[i,i] = .5
    else:
        whole_dist_avg[i,i] = 1
        whole_dist_map[i,i] = 1
    sum += 1
for i in range((order+1)**2):
    whole_dist[sum + i, sum+i-(order+1)] = 1
    if(i<(order+1)):
        whole_dist_avg[sum + i, sum+i-(order+1)] = .5
        whole_dist_map[sum + i, sum+i-(order+1)] = .5
        whole_dist_map[sum+i,sum+i] = .5
    else:
        whole_dist_avg[sum + i, sum+i-(order+1)] = 1
        whole_dist_map[sum+i,sum+i] = 1

In [31]:
whole_aux = np.dot(whole_dist_map, whole)

In [13]:
whole_classic = linalg.block_diag(lgg[0], lgg[0])

In [14]:
whole_classic_glob = np.dot(whole_dist.T, np.dot(whole_classic, whole_dist))
whole_hype_glob = np.dot(whole_dist.T, np.dot(whole, whole_dist))

whole_glob = np.dot(whole_dist.T, np.dot(whole, whole_dist))

In [20]:
def partial_diff_test(pointnum, point, matrix, pointdict):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0
    
    for k,v in pointdict.items():
        dx = point[0] - k[0]
        dy = point[1] - k[1]
            
        x_der += dx*res[root,v]
        xx_der += res[root,v]*dx**2/2
        y_der += dy*res[root,v]
        yy_der += dy**2/2*res[root,v]
        xy_der += dx*dy*res[root,v]
            
    print('Mass coef: {}'.format(mass_matrix[root].sum()))
    
    print('X derivative: {}'.format(x_der))
    print('Y derivative: {}'.format(y_der))
    print('XX derivative: {}'.format(xx_der))
    print('YY derivative: {}'.format(yy_der))
    print('XY_derivative: {}'.format(xy_der), end='\n\n')

In [25]:
from common import polynom_factory
import numpy as np
from itertools import product
sizes = [(0.,float(2**i)) for i in range(6)]

for size in sizes:
    mass_matrix = polynom_factory.local_funcfunc_matrix(order=2, dim=2, distribution='globatto', size=size)
    gradgrad = polynom_factory.local_gradgrad_matrix(order=2, distribution='globatto', dim=2, size=size)
    gradfunc = polynom_factory.local_gradfunc_matrix(order=2, distribution='globatto', dim=2, size=size)[1][0]
    print(size)
    for (point,root) in gradgrad[1].items():
        print(point)
        partial_diff_test(matrix=gradgrad[0], point=point, pointnum=root, pointdict=gradgrad[1])

(0.0, 1.0)
(0.0, 0.0)
Mass coef: 0.027777777777776014
X derivative: 0.16666666666666696
Y derivative: 0.16666666666666397
XX derivative: -0.0277777777777789
YY derivative: -0.02777777777777543
XY_derivative: 1.1102230246251565e-16

(0.0, 0.5)
Mass coef: 0.11111111111111106
X derivative: 0.6666666666666672
Y derivative: -3.2404634531246757e-15
XX derivative: -0.11111111111110925
YY derivative: -0.1111111111111123
XY_derivative: 1.3183898417423734e-16

(0.0, 1.0)
Mass coef: 0.02777777777777804
X derivative: 0.16666666666666624
Y derivative: -0.16666666666666072
XX derivative: -0.027777777777776534
YY derivative: -0.02777777777777462
XY_derivative: -2.42861286636753e-16

(0.5, 0.0)
Mass coef: 0.11111111111111228
X derivative: 0.0
Y derivative: 0.6666666666666696
XX derivative: -0.11111111111110916
YY derivative: -0.11111111111110894
XY_derivative: 0.0

(0.5, 0.5)
Mass coef: 0.44444444444444475
X derivative: -3.9968028886505635e-15
Y derivative: -1.021405182655144e-14
XX derivative: -0.444

XX derivative: -7.1111111111113985
YY derivative: -7.111111111110965
XY_derivative: -2.842170943040401e-14

(16.0, 8.0)
Mass coef: 28.444444444444553
X derivative: -10.666666666666611
Y derivative: -4.4853010194856324e-14
XX derivative: -28.44444444444371
YY derivative: -28.44444444444455
XY_derivative: -3.197442310920451e-13

(16.0, 16.0)
Mass coef: 7.1111111111112315
X derivative: -2.6666666666666208
Y derivative: -2.666666666666621
XX derivative: -7.111111111110636
YY derivative: -7.111111111110786
XY_derivative: 3.481659405224491e-13

(0.0, 32.0)
(0.0, 0.0)
Mass coef: 28.444444444442638
X derivative: 5.333333333333343
Y derivative: 5.333333333333247
XX derivative: -28.444444444445594
YY derivative: -28.44444444444204
XY_derivative: 1.1368683772161603e-13

(0.0, 16.0)
Mass coef: 113.77777777777773
X derivative: 21.33333333333335
Y derivative: -1.0369483049998962e-13
XX derivative: -113.77777777777587
YY derivative: -113.777777777779
XY_derivative: 1.3500311979441904e-13

(0.0, 32.0)